In [2]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
#os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac as aformer
#import src.aformer_TF as aformer
from src.layers.layers_atac import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac as training_utils


from scipy import stats



In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-15')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    #options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 8
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2022-09-14 18:40:31.046048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-14 18:40:31.060850: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-14 18:40:31.060934: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2022-09-14 18:40:31.473521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-14 18:40:31.911724: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.115.216.98:8470}
2022-09-14 18:40:31.911770: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:35779}
2022-09-14 18:40:49.465331: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.115.216.98:8470}
2022-09-14 18:40:49.465379: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:35779}
2022-0

INFO:tensorflow:Initializing the TPU system: node-15


INFO:tensorflow:Initializing the TPU system: node-15


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
with strategy.scope():

    train_steps=100#3200#5165 #320
    warmup_steps=100
    val_steps=10#3200#757 ### 5562
    num_epochs=10
    lr_base=0.001
    warmup_lr=1.0e-06

    data_it_tr_list = []
    data_it_val_list = []

    ### create dataset iterators
    heads_dict = {}
    orgs = ["hg"]
    for k, org in enumerate(orgs):
        heads_dict[org] = int(k)
        
    data_dict_tr,data_dict_val = training_utils.return_distributed_iterators(heads_dict,
                                                                             "gs://picard-testing-176520/16k_genecentered_blacklist0.50_atacnormalized/val_holdout",
                                                                             GLOBAL_BATCH_SIZE,
                                                                              16384,
                                                                              300,
                                                                              8,
                                                                              num_epochs,
                                                                              strategy,
                                                                              options)


In [4]:
with strategy.scope():
    model = aformer.aformer(kernel_transformation="softmax_kernel_transformation",
                                dropout_rate=0.25,
                                input_length=16384,
                                num_heads=8,
                                numerical_stabilizer=0.0000001,
                                nb_random_features=256,
                                hidden_size=256,
                                d_model=256,
                                norm=True,
                                dim=32,
                                max_seq_length = 512,
                                rel_pos_bins=128,
                                widening = 2, ## ratio between first and second dense layer units in transformer block
                                conv_filter_size_1_seq=15,
                                conv_filter_size_2_seq=5,
                                positional_dropout_rate=0.1,
                                transformer_depth=2,
                                momentum=0.90,
                                channels_list=[192,192,224,224,256,256], 
                                kernel_regularizer=0.0000001,
                                bottleneck_units_tf=64,
                                use_mask_pos=False,
                                use_rot_emb=True,
                                heads_dict=heads_dict)
    

In [5]:
with strategy.scope():
    scheduler= tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1.0e-04,
        decay_steps=1000, alpha=(5.0e-08 / 5.0e-04))
    scheduler=WarmUp(initial_learning_rate=1.0e-04,
                                 warmup_steps=150,
                                 decay_schedule_fn=scheduler)


    optimizer = tfa.optimizers.AdamW(learning_rate=scheduler,
                                     beta_1=0.9,
                                     beta_2=0.999,
                                     weight_decay=0.01)

    optimizer=tfa.optimizers.Lookahead(optimizer,
                                       sync_period=6,
                                       slow_step_size=0.5)
    

In [6]:
with strategy.scope():
    metric_dict = {}
    train_step, val_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                 optimizer,
                                                                                 strategy,
                                                                                 metric_dict, 
                                                                                 train_steps,
                                                                                 val_steps,
                                                                                 GLOBAL_BATCH_SIZE,
                                                                                 0.2,
                                                                                 True,
                                                                                 freq_limit=5000,
                                                                                 fourier_loss_scale=1.0) # last is uncropped length


In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 15):
        start = time.time()
        #if epoch_i > 2 : 
        lr, it = train_step(data_dict_tr['hg'])

                   #data_dict_tr['mm'],
                   #data_dict_tr['rm'])
        print('hg_train_loss: ' + str(metric_dict['hg_tr'].result().numpy()))
        
        #print('hg_lr: ' + str(lr.numpy()))
        #print('hg_it: ' + str(it.numpy()))
        
        val_step(data_dict_val['hg'])
  
        val_losses.append(metric_dict['hg_val'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['hg_val'].result().numpy()))
        print('hg_val_pearson: ' + str(metric_dict['hg_corr_stats'].result()['pearsonR'].numpy()))
        print('hg_val_R2: ' + str(metric_dict['hg_corr_stats'].result()['R2'].numpy()))

        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))

        for key, item in metric_dict.items():
            item.reset_state()

hg_train_loss: 0.42095244
hg_val_loss: 0.23478432
hg_val_pearson: 0.42962533
hg_val_R2: 0.0018172779
completed epoch 1
duration(mins): 2.1150352120399476
patience counter at: 0
hg_train_loss: 0.359233
hg_val_loss: 0.21166821
hg_val_pearson: 0.45217815
hg_val_R2: 0.021625359
completed epoch 2
duration(mins): 1.3981056809425354
patience counter at: 0
hg_train_loss: 0.42707747
hg_val_loss: 0.1626913
hg_val_pearson: 0.45706415
hg_val_R2: 0.029384997
completed epoch 3
duration(mins): 1.3991238872210185
patience counter at: 0
hg_train_loss: 0.49718848
hg_val_loss: 0.18476918
hg_val_pearson: 0.37506136
hg_val_R2: 0.0837932
completed epoch 4
duration(mins): 1.3869090358416238
patience counter at: 0
hg_train_loss: 0.56810826
hg_val_loss: 0.222676
hg_val_pearson: 0.42043868
hg_val_R2: -0.031031812
completed epoch 5
duration(mins): 1.3914875348409017
patience counter at: 0
hg_train_loss: 0.6076771
hg_val_loss: 0.2234356
hg_val_pearson: 0.41815025
hg_val_R2: -0.013806015
completed epoch 6
duration

In [ ]:
test = tf.constant([[[1,2],[3,4]]])

In [ ]:
test

In [19]:
test1 = tf.constant([[[1,2],[3,4],[5,6]]])

In [20]:
test1

<tf.Tensor: shape=(1, 3, 2), dtype=int32, numpy=
array([[[1, 2],
        [3, 4],
        [5, 6]]], dtype=int32)>

In [24]:
tf.concat([test,test1],1)

<tf.Tensor: shape=(1, 5, 2), dtype=int32, numpy=
array([[[1, 2],
        [3, 4],
        [1, 2],
        [3, 4],
        [5, 6]]], dtype=int32)>

In [5]:
def deserialize(serialized_example):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
      'tss_mask': tf.io.FixedLenFeature([], tf.string),
      'gene_name': tf.io.FixedLenFeature([], tf.string)
    }

    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    #print(sequence)
    sequence = tf.reshape(sequence, (196618, 4))
    #print(sequence)
    sequence = tf.cast(sequence, tf.float32)
    #print(sequence)

    target = tf.io.decode_raw(example['target'], tf.float16)

    target = tf.reshape(target,
                        (1536, 638))
    target = tf.cast(target, tf.float32)

    tss_mask = tf.ensure_shape(tf.io.parse_tensor(example['tss_mask'],
                                              out_type=tf.int32),
                           [1536,1])

    return {'sequence': sequence,
            'target': target,
            'tss_mask': tss_mask,
            'gene_name': example['gene_name']}

dataset = tf.data.TFRecordDataset("gs://genformer_data/data/human_genecentered_validation/tfrecords/valid-0.tfr",
                                compression_type='ZLIB',
                                num_parallel_reads=4)
dataset = dataset.map(lambda record: deserialize(record),
                    num_parallel_calls=4)
iter_data = iter(dataset)

next(iter_data)

{'sequence': <tf.Tensor: shape=(196618, 4), dtype=float32, numpy=
 array([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]], dtype=float32)>,
 'target': <tf.Tensor: shape=(1536, 638), dtype=float32, numpy=
 array([[0.9145508 , 0.02076721, 0.4880371 , ..., 0.        , 0.3100586 ,
         0.3515625 ],
        [2.7988281 , 0.        , 0.        , ..., 0.        , 1.8173828 ,
         3.3164062 ],
        [1.6035156 , 0.7783203 , 0.        , ..., 0.        , 1.0400391 ,
         1.5097656 ],
        ...,
        [0.        , 0.        , 0.9321289 , ..., 0.32641602, 0.        ,
         0.        ],
        [0.28588867, 0.        , 0.        , ..., 0.5761719 , 0.        ,
         0.        ],
        [0.6435547 , 0.08062744, 0.04122925, ..., 0.        , 0.        ,
         0.07501221]], dtype=float32)>,
 'tss_mask': <tf.Tensor: shape=(1536, 1), dtype=int32, numpy=
 array([[0],
 

In [6]:
out = next(iter_data)

In [7]:
tf.reduce_sum(out['tss_mask'])

<tf.Tensor: shape=(), dtype=int32, numpy=6>

In [8]:
zero = tf.constant(0, dtype=tf.int32)
where = tf.not_equal(out['tss_mask'], zero)

In [10]:
indices = tf.where(where)
indices

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[4044,    0],
       [4045,    0],
       [4046,    0]])>

In [11]:
4045 * 2 

8090

In [12]:
seq_rc = tf.gather(one_hot_encode('ATGC'), [3, 2, 1, 0], axis=-1)
seq_rc = tf.reverse(seq_rc, axis=[0])
seq_rc

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)>